<a href="https://colab.research.google.com/github/marceldavisinc-spec/Nova_pwa/blob/main/Nova_Infinty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ============================
#  NOVA ♾️ BIG BANG CORE (Colab)
#  One cell to install, wire, fetch, model, backtest, export.
#  Author: You + Nova (GPT-5 Thinking)
# ============================

# ---------- 0) RUNTIME SWITCHES ----------
NOVA_SETTINGS = {
    "SPORTS": ["baseball_mlb", "americanfootball_nfl", "basketball_nba", "icehockey_nhl", "basketball_wnba"],
    "BOOKMAKERS": ["draftkings","fanduel","betmgm","caesars"],
    "ODDS_MARKETS": ["h2h","totals","spreads"],   # keep it simple & available on free plans
    "BACKTEST_DAYS": 14,                           # change to 730 for deep two-year pulls (heavy)
    "ODDS_REGION": "us",
    "ODDS_FORMAT": "american",
    "DATE_FMT": "iso",
    "FETCH_STATCAST": True,                        # MLB only (via pybaseball). If it rate-limits, set False.
    "BUILD_SAMPLE_PICKS": True,                    # See Nova's suggested picks at the end
    "RANDOM_SEED": 42
}

# ---------- 1) API KEYS (wired per your request) ----------
ODDS_API_KEY     = "c2cc8ec16eed6ad0628674ada556c94d"     # OddsAPI
SPORTSDATAIO_KEY = "aab029af73014a68bc83a0ecc4b82561"     # SportsDataIO

# ---------- 2) INSTALL DEPS ----------
import sys, subprocess, os, json, time, math, random, textwrap, datetime as dt
def pipi(pkgs): subprocess.run([sys.executable, "-m", "pip", "install", "--quiet", "--upgrade"] + pkgs, check=False)
pipi(["requests","pandas","numpy","python-dateutil","pybaseball==2.2.7","ratelimit","tenacity","pytz"])

import requests
import pandas as pd
import numpy as np
from dateutil import tz
from dateutil.relativedelta import relativedelta
from ratelimit import limits, sleep_and_retry
from tenacity import retry, stop_after_attempt, wait_exponential
import pytz
random.seed(NOVA_SETTINGS["RANDOM_SEED"])

# ---------- 3) IO UTILS ----------
OUT_DIR = "/content/nova_out"
os.makedirs(OUT_DIR, exist_ok=True)

def save_csv(df: pd.DataFrame, name: str):
    if df is None or len(df)==0: return None
    path = os.path.join(OUT_DIR, f"{name}.csv")
    df.to_csv(path, index=False)
    return path

def save_json(obj, name: str):
    path = os.path.join(OUT_DIR, f"{name}.json")
    with open(path, "w") as f:
        json.dump(obj, f, indent=2, default=str)
    return path

def now_et():
    return dt.datetime.now(pytz.timezone("US/Eastern"))

def dstr(d): return d.strftime("%Y-%m-%d")

# ---------- 4) ODDS API HELPERS ----------
ODDS_BASE = "https://api.the-odds-api.com/v4"

def odds_url(path, params=None):
    params = params or {}
    params["apiKey"] = ODDS_API_KEY
    return f"{ODDS_BASE}{path}", params

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=1, max=8))
def get_json(url, params=None):
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    return r.json(), r.headers

def list_sports():
    url, params = odds_url("/sports", {"all": "true"})
    try:
        data, headers = get_json(url, params)
        df = pd.DataFrame(data)
        save_csv(df, "oddsapi_sports")
        return df
    except Exception as e:
        print("list_sports() error:", e)
        return pd.DataFrame()

def fetch_odds_for_sport(sport_key, markets=None, bookmakers=None, region="us", fmt="american", date_fmt="iso"):
    markets = markets or NOVA_SETTINGS["ODDS_MARKETS"]
    bookmakers = bookmakers or NOVA_SETTINGS["BOOKMAKERS"]
    all_rows = []
    for m in markets:
        url, params = odds_url(
            f"/sports/{sport_key}/odds",
            {
                "regions": region,
                "markets": m,
                "oddsFormat": fmt,
                "dateFormat": date_fmt,
                "bookmakers": ",".join(bookmakers)
            }
        )
        try:
            data, headers = get_json(url, params)
            for game in data:
                gid = game.get("id")
                commence_time = game.get("commence_time")
                home = game.get("home_team")
                away = game.get("away_team")
                for bk in game.get("bookmakers", []):
                    bk_name = bk.get("key")
                    for mk in bk.get("markets", []):
                        if mk.get("key") != m:
                            continue
                        for outc in mk.get("outcomes", []):
                            row = {
                                "sport_key": sport_key,
                                "market": m,
                                "game_id": gid,
                                "commence_time": commence_time,
                                "bookmaker": bk_name,
                                "name": outc.get("name"),
                                "price": outc.get("price"),
                                "point": outc.get("point"),
                                "home_team": home,
                                "away_team": away
                            }
                            all_rows.append(row)
        except Exception as e:
            print(f"fetch_odds_for_sport({sport_key}, {m}) error:", e)
            continue
    df = pd.DataFrame(all_rows)
    if len(df):
        df["implied_prob"] = df["price"].apply(lambda x: (abs(x)/(abs(x)+100)) if x<0 else (100/(x+100)) if pd.notna(x) else np.nan)
    return df

# ---------- 5) SPORTSDATAIO (basic pulls + MLB injuries) ----------
SDIO_BASE_NFL = "https://api.sportsdata.io/v3/nfl"
SDIO_BASE_MLB = "https://api.sportsdata.io/v3/mlb"
SDIO_BASE_WNBA = "https://api.sportsdata.io/v3/wnba"
SDIO_BASE_NBA = "https://api.sportsdata.io/v3/nba"
SDIO_BASE_NHL = "https://api.sportsdata.io/v3/nhl"

def sdio_get(url):
    r = requests.get(url, headers={"Ocp-Apim-Subscription-Key": SPORTSDATAIO_KEY}, timeout=30)
    r.raise_for_status()
    return r.json()

def sdio_nfl_season(year=2024, seasontype="PRE"):
    # Example schedules endpoint
    url = f"{SDIO_BASE_NFL}/scores/json/Schedules/{year}{seasontype}"
    try:
        return pd.DataFrame(sdio_get(url))
    except Exception as e:
        print("sdio_nfl_season error:", e)
        return pd.DataFrame()

def sdio_mlb_injuries():
    url = f"{SDIO_BASE_MLB}/scores/json/Injuries"
    try:
        return pd.DataFrame(sdio_get(url))
    except Exception as e:
        print("sdio_mlb_injuries error:", e)
        return pd.DataFrame()

# ---------- 6) PYBASEBALL (Statcast/season stats) ----------
from pybaseball import batting_stats, pitching_stats
def mlb_batting_stats(year=None):
    if year is None:
        year = now_et().year
    try:
        df = batting_stats(year, year, qual=0)
        return df
    except Exception as e:
        print("batting_stats error:", e)
        return pd.DataFrame()

def mlb_pitching_stats(year=None):
    if year is None:
        year = now_et().year
    try:
        df = pitching_stats(year, year, qual=0)
        return df
    except Exception as e:
        print("pitching_stats error:", e)
        return pd.DataFrame()

# ---------- 7) SIMPLE NOVA SIGNALS ----------
def team_form_from_odds(df_odds: pd.DataFrame):
    # Heuristic: aggregate best-line implied probs per side → normalize → pseudo “strength”
    if df_odds is None or len(df_odds)==0: return pd.DataFrame()
    h2h = df_odds[df_odds["market"]=="h2h"].copy()
    if len(h2h)==0: return pd.DataFrame()
    # best price per team side:
    grp = h2h.groupby(["sport_key","game_id","name"]).agg(
        best_price=("price", lambda s: s.dropna().min() if len(s.dropna()) else np.nan)
    ).reset_index()
    grp["implied_best"] = grp["best_price"].apply(lambda x: (abs(x)/(abs(x)+100)) if x<0 else (100/(x+100)) if pd.notna(x) else np.nan)
    # pivot to home/away if we can
    meta = h2h[["sport_key","game_id","home_team","away_team"]].drop_duplicates()
    out = grp.merge(meta, on=["sport_key","game_id"], how="left")
    # label side
    out["side"] = np.where(out["name"]==out["home_team"], "HOME", np.where(out["name"]==out["away_team"], "AWAY","NEUTRAL"))
    return out

def mlb_pitcher_edge(pitch_df: pd.DataFrame, odds_df: pd.DataFrame):
    # Map starters by name (DK/odds sometimes list team only, so this is best-effort)
    # We'll create a generic team-level pitching quality by averaging SP ERA/xFIP among likely starters — placeholder
    if pitch_df is None or len(pitch_df)==0 or odds_df is None or len(odds_df)==0: return pd.DataFrame()
    keep = pitch_df[["Name","Team","ERA","xFIP"]].copy()
    keep = keep.rename(columns={"Name":"pitcher","Team":"team"})
    # Reduce to team averages
    team_pitch = keep.groupby("team").agg(ERA=("ERA","mean"), xFIP=("xFIP","mean")).reset_index()
    # Try to align team names with odds team tokens crudely (strip city nicknames)
    def norm_team(t):
        return str(t).upper().replace(" ","").replace(".","")
    team_pitch["team_norm"] = team_pitch["team"].apply(norm_team)

    games = odds_df[odds_df["sport_key"]=="baseball_mlb"][["game_id","home_team","away_team"]].drop_duplicates().copy()
    games["home_norm"] = games["home_team"].apply(norm_team)
    games["away_norm"] = games["away_team"].apply(norm_team)

    merged = games.merge(team_pitch, left_on="home_norm", right_on="team_norm", how="left") \
                  .rename(columns={"ERA":"home_ERA","xFIP":"home_xFIP"}) \
                  .drop(columns=["team","team_norm"])
    merged = merged.merge(team_pitch, left_on="away_norm", right_on="team_norm", how="left") \
                  .rename(columns={"ERA":"away_ERA","xFIP":"away_xFIP"}) \
                  .drop(columns=["team","team_norm"])
    # Edge: lower xFIP wins; convert to pseudo prob
    merged["pitch_edge"] = (merged["away_xFIP"] - merged["home_xFIP"])  # >0 favors HOME
    # map to 0-1
    merged["pitch_edge_prob_home"] = 1/(1+np.exp(-merged["pitch_edge"]))
    merged["pitch_edge_prob_away"] = 1 - merged["pitch_edge_prob_home"]
    return merged

def combine_signals(odds_sig: pd.DataFrame, pitch_sig: pd.DataFrame):
    if odds_sig is None or len(odds_sig)==0: return pd.DataFrame()
    base = odds_sig.copy()
    # fold pitching edge if present
    if pitch_sig is not None and len(pitch_sig):
        base = base.merge(pitch_sig[["game_id","pitch_edge_prob_home","pitch_edge_prob_away"]], on="game_id", how="left")
        base["pitch_edge_prob"] = np.where(
            base["side"]=="HOME", base["pitch_edge_prob_home"],
            np.where(base["side"]=="AWAY", base["pitch_edge_prob_away"], np.nan)
        )
    # Final blended score: 60% market-implied (best line), 40% pitching edge (if exist)
    base["nova_score"] = 0.6*base["implied_best"].fillna(0.5) + 0.4*base["pitch_edge_prob"].fillna(0.5)
    base["nova_conf"] = (base["nova_score"]-0.5)*2  # -1..+1 centered
    return base.sort_values(["sport_key","game_id","nova_score"], ascending=[True,True,False])

# ---------- 8) BACKTEST (shallow) ----------
def backtest_stub(odds_df: pd.DataFrame, days: int = 14):
    # Without paid results endpoints, we simulate a sanity check:
    # If consensus implied is extreme (>= 0.65) and Nova score agrees (>= 0.60), mark as "green".
    # This is a placeholder so you can swap in real result joins later.
    if odds_df is None or len(odds_df)==0:
        return pd.DataFrame()
    h2h = odds_df[odds_df["market"]=="h2h"].copy()
    grp = h2h.groupby(["sport_key","game_id","name"]).agg(
        best_price=("price", lambda s: s.dropna().min() if len(s.dropna()) else np.nan)
    ).reset_index()
    grp["implied_best"] = grp["best_price"].apply(lambda x: (abs(x)/(abs(x)+100)) if x<0 else (100/(x+100)) if pd.notna(x) else np.nan)
    grp["green_flag"] = grp["implied_best"] >= 0.65
    return grp

# ---------- 9) MAIN RUN ----------
print("⏳ Nova Big Bang starting…")
sports_df = list_sports()
if len(sports_df):
    print(f"Sports available (sample): {sports_df['key'].head(10).tolist()}")

# Fetch odds for all target sports
all_odds = []
for s in NOVA_SETTINGS["SPORTS"]:
    df = fetch_odds_for_sport(
        s,
        markets=NOVA_SETTINGS["ODDS_MARKETS"],
        bookmakers=NOVA_SETTINGS["BOOKMAKERS"],
        region=NOVA_SETTINGS["ODDS_REGION"],
        fmt=NOVA_SETTINGS["ODDS_FORMAT"],
        date_fmt=NOVA_SETTINGS["DATE_FMT"]
    )
    if len(df):
        print(f"✔ Odds pulled: {s} — {len(df)} rows")
        save_csv(df, f"odds_{s}")
        all_odds.append(df)
    else:
        print(f"⚠ No odds returned: {s}")
odds_df = pd.concat(all_odds, ignore_index=True) if len(all_odds) else pd.DataFrame()
save_csv(odds_df, "odds_all_sports")

# MLB injuries via SportsDataIO
mlb_inj_df = sdio_mlb_injuries()
save_csv(mlb_inj_df, "mlb_injuries_sdio")

# MLB stat blocks
bat_df = mlb_batting_stats()
pit_df = mlb_pitching_stats()
save_csv(bat_df, "mlb_batting_stats_season")
save_csv(pit_df, "mlb_pitching_stats_season")

# Build signals
odds_sig = team_form_from_odds(odds_df)
pit_sig  = mlb_pitcher_edge(pit_df, odds_df) if NOVA_SETTINGS["FETCH_STATCAST"] else pd.DataFrame()
signals  = combine_signals(odds_sig, pit_sig)
save_csv(signals, "nova_signals")

# Backtest stub
bt = backtest_stub(odds_df, NOVA_SETTINGS["BACKTEST_DAYS"])
save_csv(bt, "backtest_stub")

# ---------- 10) SAMPLE PICKS (Nova quick pass) ----------
def pick_builder(signals: pd.DataFrame, top_n_per_sport=6, floor=0.58):
    if signals is None or len(signals)==0:
        return pd.DataFrame()
    picks = []
    for sport in signals["sport_key"].unique():
        sub = signals[signals["sport_key"]==sport].copy()
        sub = sub.sort_values("nova_score", ascending=False)
        sub = sub[sub["nova_score"]>=floor].head(top_n_per_sport)
        picks.append(sub)
    if len(picks)==0: return pd.DataFrame()
    picks_df = pd.concat(picks, ignore_index=True)
    # label green/yellow/red
    def tag(c):
        if c >= 0.66: return "GREEN"
        if c >= 0.58: return "YELLOW"
        return "RED"
    picks_df["nova_tag"] = picks_df["nova_score"].apply(tag)
    # collapse to game + side string
    picks_df["leg"] = picks_df.apply(lambda r: f"{r['sport_key']} | {r['name']} (vs {r['home_team'] if r['name']!=r['home_team'] else r['away_team']})", axis=1)
    return picks_df[["sport_key","game_id","side","name","nova_score","nova_conf","nova_tag","leg"]]

sample_picks = pick_builder(signals, top_n_per_sport=6, floor=0.58) if NOVA_SETTINGS["BUILD_SAMPLE_PICKS"] else pd.DataFrame()
save_csv(sample_picks, "nova_sample_picks")

# ---------- 11) SUMMARY ----------
summary = {
    "runtime_et": str(now_et()),
    "settings": NOVA_SETTINGS,
    "exports": {
        "sports_list_csv": "oddsapi_sports.csv",
        "all_odds_csv": "odds_all_sports.csv",
        "mlb_injuries_csv": "mlb_injuries_sdio.csv",
        "mlb_batting_csv": "mlb_batting_stats_season.csv",
        "mlb_pitching_csv": "mlb_pitching_stats_season.csv",
        "nova_signals_csv": "nova_signals.csv",
        "backtest_stub_csv": "backtest_stub.csv",
        "sample_picks_csv": "nova_sample_picks.csv"
    }
}
save_json(summary, "nova_summary")

print("\n✅ NOVA BIG BANG DONE.")
print(f"📂 Exports in: {OUT_DIR}")
if len(sample_picks):
    print("\n⭐ Sample Picks (top rows):")
    display(sample_picks.head(20))
else:
    print("\n(No sample picks met the floor; raise odds coverage or lower floor.)")


⏳ Nova Big Bang starting…
Sports available (sample): ['americanfootball_cfl', 'americanfootball_ncaaf', 'americanfootball_ncaaf_championship_winner', 'americanfootball_nfl', 'americanfootball_nfl_preseason', 'americanfootball_nfl_super_bowl_winner', 'americanfootball_ufl', 'aussierules_afl', 'baseball_kbo', 'baseball_milb']
✔ Odds pulled: baseball_mlb — 254 rows
✔ Odds pulled: americanfootball_nfl — 2014 rows
✔ Odds pulled: basketball_nba — 306 rows
✔ Odds pulled: icehockey_nhl — 162 rows
✔ Odds pulled: basketball_wnba — 94 rows
sdio_mlb_injuries error: 404 Client Error:  for url: https://api.sportsdata.io/v3/mlb/scores/json/Injuries

✅ NOVA BIG BANG DONE.
📂 Exports in: /content/nova_out

⭐ Sample Picks (top rows):


,sport_key,game_id,side,name,nova_score,nova_conf,nova_tag,leg
0,americanfootball_nfl,d16a43752aaac66e9ff8768d8590b662,HOME,Buffalo Bills,0.722581,0.445161,GREEN,americanfootball_nfl | Buffalo Bills (vs New O...
1,americanfootball_nfl,553edb8bf2452482b37dde58e832a6fb,HOME,Baltimore Ravens,0.722581,0.445161,GREEN,americanfootball_nfl | Baltimore Ravens (vs Cl...
2,americanfootball_nfl,7159d60c7a31c7d593167637723d7663,HOME,Baltimore Ravens,0.717241,0.434483,GREEN,americanfootball_nfl | Baltimore Ravens (vs Ne...
3,americanfootball_nfl,bcb1f89cf378a586097c50585e53acce,HOME,Detroit Lions,0.707692,0.415385,GREEN,americanfootball_nfl | Detroit Lions (vs Cleve...
4,americanfootball_nfl,a8ebe01e05675c109b53a97f653f8653,HOME,Philadelphia Eagles,0.707692,0.415385,GREEN,americanfootball_nfl | Philadelphia Eagles (vs...
5,americanfootball_nfl,813dab51d999b87c7968ac4890965f73,HOME,Philadelphia Eagles,0.707692,0.415385,GREEN,americanfootball_nfl | Philadelphia Eagles (vs...
6,baseball_mlb,89047d113644dcf45e4495dcc09fb0d4,HOME,Oakland Athletics,0.794059,0.588119,GREEN,baseball_mlb | Oakland Athletics (vs Los Angel...
7,baseball_mlb,7512790901621f91bf67069885402506,HOME,Boston Red Sox,0.628571,0.257143,YELLOW,baseball_mlb | Boston Red Sox (vs Miami Marlins)
8,baseball_mlb,d4f35e82f8213058643776e352cbe252,HOME,Chicago Cubs,0.606452,0.212903,YELLOW,baseball_mlb | Chicago Cubs (vs Pittsburgh Pir...
9,baseball_mlb,57ada182eefa02856b12313f002a0b34,AWAY,Philadelphia Phillies,0.585714,0.171429,YELLOW,baseball_mlb | Philadelphia Phillies (vs Washi...


In [3]:
# ================= NOVA PATCH: MLB Injuries (drop-in) =================
# Safe MLB injuries loader for SportsDataIO. Works even on free plans
# (returns an empty, schema-stable DataFrame instead of crashing).

import os, time, json, requests
import pandas as pd
from datetime import datetime

def _log(msg):
    print(msg, flush=True)

def _fetch_json(url, headers=None, params=None, timeout=20):
    try:
        r = requests.get(url, headers=headers or {}, params=params or {}, timeout=timeout)
        if r.status_code == 404:
            # Free plan or endpoint not enabled — treat as empty
            return None, 404, f"404 Not Found for {url}"
        r.raise_for_status()
        return r.json(), r.status_code, None
    except requests.exceptions.RequestException as e:
        return None, None, str(e)

# --- key resolver: finds your SDIO key from globals or env so you don't edit the cell
def _get_sdio_key():
    # try common global variable names you may already have in Nova Core
    for k in ["SPORTSDATAIO_KEY", "SPORTS_DATAIO_KEY", "SDIO_KEY", "SPORTSDATA_KEY"]:
        if k in globals() and isinstance(globals()[k], str) and globals()[k]:
            return globals()[k]
    # try environment
    for k in ["SPORTSDATAIO_KEY", "SPORTS_DATAIO_KEY", "SDIO_KEY", "SPORTSDATA_KEY"]:
        v = os.environ.get(k, "")
        if v:
            return v
    return ""

def sdio_mlb_injuries(api_key: str, season: str | None = None):
    """
    Returns a pandas.DataFrame of MLB injuries if your SportsDataIO key
    has access. On free plans (404) it returns an empty DataFrame and a
    status dict so the pipeline never breaks.
    """
    if not api_key:
        _log("sdio_mlb_injuries: no API key provided; returning empty.")
        return pd.DataFrame(), {"ok": False, "why": "missing_key", "rows": 0}

    if season is None:
        season = str(datetime.utcnow().year)  # e.g., '2025'

    base = "https://api.sportsdata.io/v3/mlb/scores/json"
    candidates = [
        f"{base}/Injuries/{season}",  # season-scoped
        f"{base}/Injuries",           # generic
    ]
    headers = {"Ocp-Apim-Subscription-Key": api_key}

    for url in candidates:
        data, status, err = _fetch_json(url, headers=headers)
        if status == 200 and isinstance(data, list):
            df = pd.DataFrame(data)
            rows = len(df)
            _log(f"✓ sdio_mlb_injuries: pulled {rows} rows from {url}")
            return df, {"ok": True, "why": "ok", "rows": rows, "url": url}
        elif status == 404:
            _log(f"• sdio_mlb_injuries 404 (free plan / not enabled): {url}")
            continue
        elif err:
            _log(f"• sdio_mlb_injuries error: {err}")
            continue

    # Fallback: empty schema so downstream joins won't break
    empty_cols = [
        "InjuryID","PlayerID","Name","Team","Position","InjuryStatus",
        "InjuryBodyPart","InjuryNotes","Updated","Practice","PracticeDescription",
        "ReturnDate"
    ]
    df_empty = pd.DataFrame(columns=empty_cols)
    _log("⨯ sdio_mlb_injuries: returning empty (endpoint unavailable on this plan).")
    return df_empty, {"ok": False, "why": "unavailable_or_free_plan", "rows": 0}

def load_mlb_injuries_into(data_store: dict, api_key: str):
    """
    Pulls MLB injuries safely and parks them in the shared `data_store`
    under keys: 'mlb_injuries' and 'mlb_injuries_meta'.
    Also prints a one-line summary.
    """
    df, meta = sdio_mlb_injuries(api_key)
    data_store["mlb_injuries"] = df
    data_store["mlb_injuries_meta"] = meta
    _log(f"MLB injuries: ok={meta.get('ok', False)} rows={meta.get('rows', 0)}")
    return df, meta

# ---- integrate with Nova's shared DATA dict and run now
if "DATA" not in globals() or not isinstance(globals().get("DATA"), dict):
    DATA = {}

_sdio_key = _get_sdio_key()
_df_inj, _meta_inj = load_mlb_injuries_into(DATA, _sdio_key)

# quick peek so you can verify without other cells
try:
    from IPython.display import display
    display(_df_inj.head(10))
except Exception:
    print(_df_inj.head(10))
# ====================================================================


• sdio_mlb_injuries 404 (free plan / not enabled): https://api.sportsdata.io/v3/mlb/scores/json/Injuries/2025
• sdio_mlb_injuries 404 (free plan / not enabled): https://api.sportsdata.io/v3/mlb/scores/json/Injuries
⨯ sdio_mlb_injuries: returning empty (endpoint unavailable on this plan).
MLB injuries: ok=False rows=0


,InjuryID,PlayerID,Name,Team,Position,InjuryStatus,InjuryBodyPart,InjuryNotes,Updated,Practice,PracticeDescription,ReturnDate
